In [27]:
import numpy as np
import matplotlib.pyplot as plt
import SimpleITK as sitk
import os
import pydicom
from registration_utilities import *

%matplotlib auto

Using matplotlib backend: MacOSX


In [41]:
def remove_keymap_conflicts(new_keys_set):
    for prop in plt.rcParams:
        if prop.startswith('keymap.'):
            keys = plt.rcParams[prop]
            remove_list = set(keys) & new_keys_set
            for key in remove_list:
                keys.remove(key)
                
def multi_slice_viewer(volume, mask=None):
    remove_keymap_conflicts({'j', 'k'})
    fig, ax = plt.subplots()
    
    if mask is not None:
        mask = mask*np.max(volume)/2

    ax.volume = volume + mask
        
    ax.index = volume.shape[2] // 2
    ax.imshow(volume[:,:,ax.index])
    
#    ax.imshow(mask[:,:,ax.index], alpha=0.5)
        
    fig.canvas.mpl_connect('key_press_event', process_key)

def process_key(event):
    fig = event.canvas.figure
    ax = fig.axes[0]
    if event.key == 'j':
        previous_slice(ax)
    elif event.key == 'k':
        next_slice(ax)
    fig.canvas.draw()

def previous_slice(ax):
    volume = ax.volume
#    mask = ax.mask
    ax.index = (ax.index - 1) % volume.shape[2]  # wrap around using %
    ax.images[0].set_array(volume[:,:, ax.index])
#    ax.images[1].set_array(mask[:,:, ax.index])

def next_slice(ax):
    volume = ax.volume
#    mask = ax.mask
    ax.index = (ax.index + 1) % volume.shape[2]
    ax.images[0].set_array(volume[:,:,ax.index])
#    ax.images[1].set_array(mask[:,:, ax.index])

In [28]:
DATADIR_1 = '/Users/riccardobusetti/Desktop/tmp_processed/pa001/st000/se001'
DATADIR_2 = '/Users/riccardobusetti/Desktop/tmp_processed/pa001/st000/se000'

reader_1 = sitk.ImageSeriesReader()
reader_1.SetOutputPixelType(sitk.sitkFloat32)
dicom_names_1 = reader_1.GetGDCMSeriesFileNames(DATADIR_1)
reader_1.SetFileNames(dicom_names_1)

reader_2 = sitk.ImageSeriesReader()
reader_2.SetOutputPixelType(sitk.sitkFloat32)
dicom_names_2 = reader_2.GetGDCMSeriesFileNames(DATADIR_2)
reader_2.SetFileNames(dicom_names_2)

In [29]:
fixed_image = reader_1.Execute()
moving_image = reader_2.Execute()

moving_to_fixed = sitk.Resample(moving_image, fixed_image)

In [30]:
moving_to_fixed_np = sitk.GetArrayFromImage(moving_to_fixed)
moving_image_np = sitk.GetArrayFromImage(moving_image)
fixed_image_np = sitk.GetArrayFromImage(fixed_image)

In [31]:
plt.figure()
plt.subplot(131)
plt.imshow(fixed_image_np[190,:,:])

plt.subplot(132)
plt.imshow(moving_image_np[24,:,:])

plt.subplot(133)
plt.imshow(moving_to_fixed_np[96,:,:])

In [42]:
multi_slice_viewer(moving_image_np)


TypeError: unsupported operand type(s) for +: 'float' and 'NoneType'

In [33]:
print(fixed_image_np.shape)
print(moving_image_np.shape)
print(moving_to_fixed_np.shape)

(192, 256, 252)
(50, 192, 192)
(192, 256, 252)


In [34]:
initial_transform = sitk.CenteredTransformInitializer(fixed_image, 
                                                      moving_to_fixed, 
                                                      sitk.Euler3DTransform(), 
                                                      sitk.CenteredTransformInitializerFilter.GEOMETRY)

In [35]:
display_registration_results(fixed_image, moving_to_fixed, initial_transform)

interactive(children=(IntSlider(value=95, description='image_z', max=191), FloatSlider(value=0.5, description=…

In [36]:
print(initial_transform)

itk::simple::Transform
 Euler3DTransform (0x7fdbf666aae0)
   RTTI typeinfo:   itk::Euler3DTransform<double>
   Reference Count: 1
   Modified Time: 18277
   Debug: Off
   Object Name: 
   Observers: 
     none
   Matrix: 
     1 0 0 
     0 1 0 
     0 0 1 
   Offset: [0, 0, 0]
   Center: [1.75208, 16.1299, 2.24798]
   Translation: [0, 0, 0]
   Inverse: 
     1 0 0 
     0 1 0 
     0 0 1 
   Singular: 0
   Euler's angles: AngleX=0 AngleY=0 AngleZ=0
   m_ComputeZYX = 0



In [37]:
registration_method = sitk.ImageRegistrationMethod()

registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.01)

registration_method.SetInterpolator(sitk.sitkLinear)

registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=60)
registration_method.SetOptimizerScalesFromPhysicalShift() 

registration_method.SetInitialTransform(initial_transform, inPlace=False)

registration_method.AddCommand(sitk.sitkStartEvent, metric_start_plot)
registration_method.AddCommand(sitk.sitkEndEvent, metric_end_plot)
registration_method.AddCommand(sitk.sitkIterationEvent, 
                               lambda: metric_plot_values(registration_method))

final_transform_v1 = registration_method.Execute(fixed_image, moving_to_fixed)

In [38]:
print(final_transform_v1)

itk::simple::Transform
 CompositeTransform (0x7fdbf67871a0)
   RTTI typeinfo:   itk::CompositeTransform<double, 3u>
   Reference Count: 1
   Modified Time: 534909
   Debug: Off
   Object Name: 
   Observers: 
     none
   Transforms in queue, from begin to end:
   >>>>>>>>>
   Euler3DTransform (0x7fdbf665bf60)
     RTTI typeinfo:   itk::Euler3DTransform<double>
     Reference Count: 1
     Modified Time: 534900
     Debug: Off
     Object Name: 
     Observers: 
       none
     Matrix: 
       0.99971 -0.00765767 0.0228373 
       0.00914618 0.997791 -0.0658035 
       -0.0222829 0.0659933 0.997571 
     Offset: [-0.0812195, -2.32241, 39.6631]
     Center: [1.75208, 16.1299, 2.24798]
     Translation: [-0.153907, -2.48995, 40.6831]
     Inverse: 
       0.99971 0.00914618 -0.0222829 
       -0.00765767 0.997791 0.0659933 
       0.0228373 -0.0658035 0.997571 
     Singular: 0
     Euler's angles: AngleX=0.0660413 AngleY=0.0223335 AngleZ=0.00767448
     m_ComputeZYX = 0
   End of Multi

In [39]:
%matplotlib auto

Using matplotlib backend: MacOSX


In [40]:
display_registration_results(fixed_image, moving_to_fixed, final_transform_v1)

interactive(children=(IntSlider(value=95, description='image_z', max=191), FloatSlider(value=0.5, description=…